## SQL PROJECT

Question 1:

Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

Answer 1:

Query: 

SELECT DISTINCT name	
FROM country_club.Facilities
WHERE membercost >0

Result:

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court

Question 2:

How many facilities do not charge a fee to members?

Answer 2:

Query:

SELECT COUNT( DISTINCT name )
FROM country_club.Facilities
WHERE membercost <=0

Result:

COUNT( DISTINCT name )
4

Question 3:

How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

Answer 3:

Query:

SELECT facid, name, membercost, monthlymaintenance
FROM country_club.Facilities
WHERE membercost <20 /100 * monthlymaintenance
AND membercost >0

Result:

facid	    name	    membercost	  monthlymaintenance	
0	    Tennis Court 1	   5.0	           200
1	    Tennis Court 2	   5.0	           200
4	    Massage Room 1	   9.9	           3000
5	    Massage Room 2	   9.9	           3000
6	    Squash Court	   3.5	           80

Question 4:

How can you retrieve the details of facilities with ID 1 and 5?
Write the query without using the OR operator. 

Answer 4:

Query: 

SELECT *
FROM country_club.Facilities
WHERE facid IN (1, 5)

Result:

facid	name	   membercost	guestcost	initialoutlay	monthlymaintenance	
1	Tennis Court 2	  5.0	      25.0	       8000	            200
5	Massage Room 2	  9.9	      80.0	       4000	            3000

Question 5:

How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question.

Answer 5:

Query:

SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance <=100
THEN 'cheap'
WHEN monthlymaintenance >100
THEN 'expensive'
ELSE NULL
END AS value
FROM country_club.Facilities


Result:

 name	           monthlymaintenance	       value	
Tennis Court 1	         200	             expensive
Tennis Court 2	         200	             expensive
Badminton Court	         50	                 cheap
Table Tennis	         10	                 cheap
Massage Room 1	         3000	             expensive
Massage Room 2	         3000	             expensive
Squash Court	         80	                 cheap
Snooker Table	         15	                 cheap
Pool Table	             15	                 cheap


Question 6:

You'd like to get the first and last name of the last member(s)
who signed up. Do not use the LIMIT clause for your solution.

Answer 6:

Query:

SELECT surname, firstname
FROM country_club.Members
WHERE joindate = (SELECT MAX( joindate )FROM country_club.Members )

Result:

surname	   firstname	
Smith	    Darren

Question 7:

How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

Answer 7:

Query:

SELECT DISTINCT CONCAT( mem.surname, ', ', mem.firstname ) AS member, fac.name AS facility
FROM country_club.Members mem
JOIN country_club.Bookings bks ON mem.memid = bks.memid
JOIN country_club.Facilities fac ON fac.facid = bks.facid
WHERE fac.name LIKE 'tennis%%'
ORDER BY member

Result:

member	                  facility	
Bader, Florence	        Tennis Court 2
Bader, Florence	        Tennis Court 1
Baker, Anne	            Tennis Court 1
Baker, Anne	            Tennis Court 2
Baker, Timothy	        Tennis Court 2
Baker, Timothy	        Tennis Court 1
Boothe, Tim	            Tennis Court 1
Boothe, Tim	            Tennis Court 2
Butters, Gerald	        Tennis Court 1
Butters, Gerald	        Tennis Court 2
Coplin, Joan	        Tennis Court 1
Crumpet, Erica	        Tennis Court 1
Dare, Nancy	            Tennis Court 2
Dare, Nancy	            Tennis Court 1
Farrell, David	        Tennis Court 1
Farrell, David	        Tennis Court 2
Farrell, Jemima	        Tennis Court 1
Farrell, Jemima	        Tennis Court 2
Genting, Matthew	    Tennis Court 1
GUEST, GUEST	        Tennis Court 2
GUEST, GUEST	        Tennis Court 1
Hunt, John	            Tennis Court 1
Hunt, John	            Tennis Court 2
Jones, David	        Tennis Court 1
Jones, David	        Tennis Court 2
Jones, Douglas	        Tennis Court 1
Joplette, Janice	    Tennis Court 1
Joplette, Janice	    Tennis Court 2
Owen, Charles	        Tennis Court 1
Owen, Charles	        Tennis Court 2



Question 8:

How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

Answer 8:

Query:

SELECT distinct fac.name as Facility, concat(mem.surname,', ', mem.firstname) as Member, 
(fac.membercost + fac.guestcost * 2) as Cost
FROM country_club.Bookings bks
JOIN country_club.Facilities fac
ON bks.facid = fac.facid
JOIN country_club.Members mem
ON bks.memid = mem.memid
WHERE bks.starttime > "2012-09-13 23:59:59" AND bks.starttime < "2012-09-15 00:00:00"
AND ( fac.membercost > 30 OR fac.guestcost > 30 )
ORDER BY Cost DESC

Result:

Facility	           Member	      Cost	
Massage Room 1	   GUEST, GUEST	      169.9
Massage Room 1	   Stibbons, Ponder	  169.9
Massage Room 1	   Genting, Matthew	  169.9
Massage Room 2	   GUEST, GUEST	      169.9
Massage Room 1	   Smith, Jack        169.9
Massage Room 1	   Farrell, Jemima	  169.9
Massage Room 1	   Tracy, Burton	  169.9
Massage Room 2	   Bader, Florence	  169.9


Question 9:

This time, produce the same result as in Q8, but using a subquery.

Answer 9:

Query:

SELECT distinct fac.name as Facility, concat(mem.surname,', ', mem.firstname) as Member, (fac.membercost + fac.guestcost * 2) as Cost
FROM country_club.Bookings bks
JOIN
-- subquery to get the facilities information
     ( SELECT facid, name, membercost, guestcost from country_club.Facilities ) fac
ON fac.facid = bks.facid
JOIN
-- subquery to get the member informaton
     ( SELECT memid, surname, firstname from country_club.Members ) mem
ON mem.memid = bks.memid
WHERE bks.starttime > "2012-09-13 23:59:59" AND bks.starttime < "2012-09-15 00:00:00"
AND ( fac.membercost > 30 OR fac.guestcost > 30 )
ORDER BY Cost DESC

Result:

Facility	           Member	      Cost	
Massage Room 1	   GUEST, GUEST	      169.9
Massage Room 1	   Stibbons, Ponder	  169.9
Massage Room 1	   Genting, Matthew	  169.9
Massage Room 2	   GUEST, GUEST	      169.9
Massage Room 1	   Smith, Jack        169.9
Massage Room 1	   Farrell, Jemima	  169.9
Massage Room 1	   Tracy, Burton	  169.9
Massage Room 2	   Bader, Florence	  169.9

Question 10:

Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

Answer 10:

Query:

select name, revenue from (
  -- Revenue = Income - Expense
  SELECT fac.facid, fac.name, (income - expense) as revenue
  from country_club.Facilities fac
  JOIN
  -- Calculate income from member bookings (Assume 1 member and 1 guest for each booking)
  (SELECT fac.facid, fac.name, COUNT(*)
 * (fac.membercost + fac.guestcost) as income,0
     FROM country_club.Bookings bks
     JOIN country_club.Facilities fac
     ON bks.facid = fac.facid
     GROUP by fac.facid, fac.name) as T1_income
  ON T1_income.facid = fac.facid
  JOIN 
  -- Calculate expense from monthlymaintenance cost over the months spanned by the entries in the Bookings table
  (SELECT fac.facid, 0, fac.monthlymaintenance * months as expense
    FROM country_club.Facilities fac
    JOIN 
    -- Determine the number of months spanned by dates in Bookings table for a given facility, 
    -- rounded to the nearest integer
    (select facid, round(DATEDIFF(maxi,mini) / 30) as months from 
      ( select fac2.facid, max(bks2.starttime) as maxi, min(bks2.starttime) as mini
           FROM country_club.Bookings bks2
           JOIN country_club.Facilities fac2
           ON fac2.facid = bks2.facid
           GROUP BY fac2.facid
       ) A1
    ) as mm
    ON 
    mm.facid = fac.facid
    ) as T1_expense
  ON T1_expense.facid = fac.facid
) B
-- Show only facilities whose revenue is less than 1000
where revenue < 1000


Result:

name	           revenue	
Massage Room 2	    978.9
